In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('m2.csv')
df.head()

,sex,suicides_no,population,suicides_pop,gdp_per_capita,lvl_gdp,GDP_for_group,Boomers,G_I_Generation,GenerationX,GenerationZ,Millenials,Silent
0,1.0,21.0,312900.0,6.71,796.0,0.0,249068400.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,16.0,308000.0,5.19,796.0,0.0,245168000.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,14.0,289700.0,4.83,796.0,0.0,230601200.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,1.0,21800.0,4.59,796.0,0.0,17352800.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.0,9.0,274300.0,3.28,796.0,0.0,218342800.0,1.0,0.0,0.0,0.0,0.0,0.0


In [3]:
X = df.drop(('sex'), axis=1)
y = df['sex']
feature_names = X.columns
print (feature_names)

Index(['suicides_no', 'population', 'suicides_pop', 'gdp_per_capita',
       'lvl_gdp', 'GDP_for_group', 'Boomers', 'G_I_Generation', 'GenerationX',
       'GenerationZ', 'Millenials', 'Silent'],
      dtype='object')


In [4]:
print (X.shape)
print (y.shape)
N, d = X.shape

(27820, 12)
(27820,)


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 17)

N_train, _ = X_train.shape 
N_test,  _ = X_test.shape 
print (N_train, N_test)

19474 8346


In [6]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

ad = AdaBoostClassifier(DecisionTreeClassifier(),
                        algorithm="SAMME", n_estimators=200)
ad.fit(X_train, y_train)
pred_a = ad.predict(X_test)

acc_a = metrics.accuracy_score(y_test, pred_a)
print (acc_a)

0.7751018451953031


In [7]:
roc_a = metrics.roc_auc_score(y_test, pred_a)
print (roc_a)

0.775087748515511


In [8]:
from xgboost import XGBClassifier

xg = XGBClassifier(learning_rate=0.1, n_estimators=140, max_depth=5,
                        min_child_weight=3, gamma=0.2, subsample=0.6, colsample_bytree=1.0,
                        objective='binary:logistic', nthread=4, scale_pos_weight=1, seed=27)
xg.fit(X_train, y_train)
pred_x = xg.predict(X_test)

acc_x = metrics.accuracy_score(y_test, pred_x)
print (acc_x)

0.7673136832015337


In [9]:
roc_x = metrics.roc_auc_score(y_test, pred_x)
print (roc_x)

0.7668998013024453


In [10]:
import lightgbm as lgb

params = {
        'task': 'train',
        'objective': 'binary',    
        'metric': 'binary_error', 
        'verbose': 1
}
gbm = lgb.LGBMClassifier(learning_rate = 0.35, metric = 'l2', 
                        n_estimators = 65)
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric=['auc', 'binary_logloss'],
early_stopping_rounds=5)
pred_g = gbm.predict(X_test)

[1]	valid_0's auc: 0.812618	valid_0's binary_logloss: 0.603954	valid_0's l2: 0.206318
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's auc: 0.826272	valid_0's binary_logloss: 0.55613	valid_0's l2: 0.185172


/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


[3]	valid_0's auc: 0.832767	valid_0's binary_logloss: 0.528542	valid_0's l2: 0.174442
[4]	valid_0's auc: 0.837634	valid_0's binary_logloss: 0.510412	valid_0's l2: 0.168204
[5]	valid_0's auc: 0.842841	valid_0's binary_logloss: 0.495228	valid_0's l2: 0.16314
[6]	valid_0's auc: 0.844695	valid_0's binary_logloss: 0.487469	valid_0's l2: 0.160976
[7]	valid_0's auc: 0.847055	valid_0's binary_logloss: 0.480712	valid_0's l2: 0.158969
[8]	valid_0's auc: 0.848439	valid_0's binary_logloss: 0.475374	valid_0's l2: 0.157486
[9]	valid_0's auc: 0.850497	valid_0's binary_logloss: 0.471007	valid_0's l2: 0.156249
[10]	valid_0's auc: 0.853018	valid_0's binary_logloss: 0.465825	valid_0's l2: 0.154695
[11]	valid_0's auc: 0.854185	valid_0's binary_logloss: 0.463104	valid_0's l2: 0.154033
[12]	valid_0's auc: 0.855891	valid_0's binary_logloss: 0.46068	valid_0's l2: 0.153251
[13]	valid_0's auc: 0.857098	valid_0's binary_logloss: 0.459417	valid_0's l2: 0.15278
[14]	valid_0's auc: 0.858481	valid_0's binary_logloss

In [11]:
acc_g = metrics.accuracy_score(y_test, pred_g)
print (acc_g)

0.7891205367840882


In [12]:
roc_g = metrics.roc_auc_score(y_test, pred_g)
print (roc_g)

0.7887680751605085


In [13]:
from catboost import CatBoostClassifier

cat = CatBoostClassifier(iterations=30, learning_rate=1, depth=6, loss_function='Logloss')
cat.fit(X_train, y_train)
pred_c = cat.predict(X_test)

0:	learn: 0.5497749	total: 107ms	remaining: 3.11s
1:	learn: 0.5352083	total: 150ms	remaining: 2.1s
2:	learn: 0.5261567	total: 187ms	remaining: 1.69s
3:	learn: 0.5190873	total: 226ms	remaining: 1.47s
4:	learn: 0.5139799	total: 269ms	remaining: 1.35s
5:	learn: 0.5100478	total: 332ms	remaining: 1.33s
6:	learn: 0.5049159	total: 374ms	remaining: 1.23s
7:	learn: 0.5018811	total: 413ms	remaining: 1.14s
8:	learn: 0.4971557	total: 452ms	remaining: 1.05s
9:	learn: 0.4949168	total: 526ms	remaining: 1.05s
10:	learn: 0.4916772	total: 589ms	remaining: 1.02s
11:	learn: 0.4901688	total: 627ms	remaining: 940ms
12:	learn: 0.4886794	total: 663ms	remaining: 867ms
13:	learn: 0.4877677	total: 722ms	remaining: 825ms
14:	learn: 0.4876165	total: 832ms	remaining: 832ms
15:	learn: 0.4843333	total: 873ms	remaining: 764ms
16:	learn: 0.4791824	total: 966ms	remaining: 739ms
17:	learn: 0.4762569	total: 1.03s	remaining: 690ms
18:	learn: 0.4760567	total: 1.09s	remaining: 630ms
19:	learn: 0.4750104	total: 1.13s	remainin

In [14]:
acc_c = metrics.accuracy_score(y_test, pred_c)
print(acc_c)

0.7588066139468008


In [15]:
roc_c = metrics.roc_auc_score(y_test, pred_c)
print (roc_c)

0.7584406836114715


In [16]:
per_a = (acc_a + roc_a) / 2
per_x = (acc_x + roc_x) / 2
per_g = (acc_g + roc_g) / 2
per_c = (acc_c + roc_c) / 2

In [17]:
t = [per_a, per_x, per_c, per_g]
t = max(t)

In [18]:
if per_a == t:
    print ('AdaBoost Classifier accuracy score:', acc_a, 'roc auc score:', roc_a)
elif per_x == t:
    print ('XGB Classifier accuracy score:', acc_x, 'roc auc score:', roc_x)
elif per_g == t:
    print ('LGBM Classifier accuracy score:', acc_g, 'roc auc score:', roc_g)
elif per_c == t:
    print ('CatBoost Classifier accuracy score:', acc_c, 'roc auc score:', roc_c)

LGBM Classifier accuracy score: 0.7891205367840882 roc auc score: 0.7887680751605085
